In [42]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve

from sklearn.metrics import confusion_matrix

### Загрузите файл classification.csv. В нем записаны истинные классы объектов выборки (колонка true) и ответы некоторого классификатора (колонка pred).

In [2]:
data = pd.read_csv('classification.csv')

In [3]:
data.head()

,true,pred
0,1,0
1,1,1
2,1,1
3,0,0
4,1,1


### Заполните таблицу ошибок классификации:

<table><tbody><tr><td></td><td>Actual Positive</td><td>Actual Negative</td></tr><tr><td>Predicted Positive</td><td>TP</td><td>FP</td></tr><tr><td>Predicted Negative</td><td>FN</td><td>TN</td></tr></tbody></table>

### Для этого подсчитайте величины TP, FP, FN и TN согласно их определениям. Например, FP — это количество объектов, имеющих класс 0, но отнесенных алгоритмом к классу 1. Ответ в данном вопросе — четыре числа через пробел.

In [12]:
conf_matrix = confusion_matrix(data['true'], data['pred'])
confusion_table = pd.DataFrame(conf_matrix, 
                               index=['Predict Positive', 'Predict Negative'],
                               columns=['Actual Positive', 'Actual Negative'])

In [13]:
confusion_table

,Actual Positive,Actual Negative
Predict Positive,64,34
Predict Negative,59,43


In [75]:
tn, fp, fn, tp = conf_matrix.ravel()
answer1 = str(tp) + ' ' + str(fp) + ' ' + str(fn) + ' ' + str(tn)

In [76]:
with open('task1.txt', 'w') as f:
    f.write(answer1)

### Посчитайте основные метрики качества классификатора:
+ Accuracy (доля верно угаданных) — sklearn.metrics.accuracy_score
+ Precision (точность) — sklearn.metrics.precision_score
+ Recall (полнота) — sklearn.metrics.recall_score
+ F-мера — sklearn.metrics.f1_score

### В качестве ответа укажите эти четыре числа через пробел.

In [29]:
answer2 = []
answer2.append(str(accuracy_score(data['true'], data['pred'])))
answer2.append(str(precision_score(data['true'], data['pred'])))
answer2.append(str(recall_score(data['true'], data['pred'])))
answer2.append(str(f1_score(data['true'], data['pred'])))

In [30]:
with open('task2.txt', 'w') as f:
    f.write(' '.join(answer2))

### Имеется четыре обученных классификатора. В файле scores.csv записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:
+ для логистической регрессии — вероятность положительного класса (колонка score_logreg)
+ для SVM — отступ от разделяющей поверхности (колонка score_svm)
+ для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn)
+ для решающего дерева — доля положительных объектов в листе (колонка score_tree)

### Загрузите этот файл.

In [31]:
data_scores = pd.read_csv('scores.csv')

In [32]:
data_scores.head()

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333
2,0,0.382315,-0.245701,0.000000,0.000000
3,1,0.506797,-0.137058,0.000000,0.105263
4,1,0.488781,-0.154148,0.000000,0.105263


### Посчитайте площадь под ROC-кривой для каждого классификатора. Какой классификатор имеет наибольшее значение метрики AUC-ROC (укажите название столбца)? Воспользуйтесь функцией sklearn.metrics.roc_auc_score.

In [77]:
true_ans = data_scores['true']
clf_scores = data_scores.drop(['true'], axis=1)

In [78]:
best_val = -1
best_clf = ''
for col in clf_scores.columns:
    score = roc_auc_score(true_ans, clf_scores[col])
    if best_val < score:
        best_val = score
        best_clf = col

In [79]:
best_clf, best_val

('score_logreg', 0.71918767507002801)

In [80]:
with open('task3.txt', 'w') as f:
    f.write(str(best_clf))

### Какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70% ?

In [72]:
best_max = -1
best_clf = ''
for col in clf_scores.columns:
    precision, recall, thresholds = precision_recall_curve(true_ans, clf_scores[col])
    # recall = [r if r >= 0.7 else 0 for r in recall]
    local_max = np.max(precision[recall >= 0.7])
    if local_max > best_max:
        best_max = local_max
        best_clf = col

In [73]:
best_clf, best_max

('score_tree', 0.6517857142857143)

In [74]:
with open('task4.txt', 'w') as f:
    f.write(str(best_clf))